# ETL del dataset lesiones
Importación de librerías 

In [80]:
import pandas as pd
import numpy as np

## Lesiones hechos

Carga del dataset

In [81]:
df_hechos = pd.read_excel('../Data/lesiones.xlsx', sheet_name = 'HECHOS', na_values=['SD','sd'])

Visualización de las primeras 5 filas

In [82]:
df_hechos.head()

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9.0,NaN,14,...,-34.559658,CICLISTA,NaN,CICLISTA-SD,NaN,NaN,NaN,NaN,x,NaN
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1.0,NaN,8,...,-34.669125,AUTO,NaN,AUTO-SD,NaN,x,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2.0,NaN,8,...,-34.677556,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2.0,NaN,7,...,-34.647349,PEATON,NaN,PEATON-SD,x,NaN,NaN,NaN,NaN,NaN
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4.0,NaN,3,...,-34.604579,AUTO,NaN,AUTO-SD,NaN,NaN,x,NaN,NaN,NaN


Información de las columnas del dataset

In [83]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23776 non-null  object 
 7   franja_hora            23776 non-null  float64
 8   direccion_normalizada  12917 non-null  object 
 9   comuna                 22770 non-null  object 
 10  tipo_calle             12740 non-null  object 
 11  otra_direccion         5490 non-null   object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

#### Elimino columnas innecesarias


Luego de analizar las columnas se determino que: 
* Las columnas `aaaa`,`mm`,`dd` hacen referencia a la información de la columna `fecha`
* La columna `hora` hacen referencia a la información de la columna `franja_hora`
* Las columnas `direccion_normalizada`, `otra_direccion` ,`calle`, `altura`, `cruce`  y `geocodificacion_CABA` ya que con usar las columnas `longitud` y `latitud` respectiva del accidente
* Las columnas `moto`,`auto`,`transporte_publico`,`camion`,`ciclista`,`participante` hacen referencia a la información de la columna `victima` y `acusado`

In [84]:
df_hechos.drop(columns=['aaaa','mm','dd','hora','direccion_normalizada','otra_direccion','calle','altura','cruce','geocodificacion_CABA','moto','auto','transporte_publico','camion','ciclista','participantes'],inplace=True)

#### Valores nulos


Conteo de nulos

In [85]:
df_hechos.isnull().sum()

id                 0
n_victimas         0
fecha              0
franja_hora        9
comuna          1015
tipo_calle     11045
longitud        1471
latutid         1471
victima        10733
acusado        15288
gravedad       23139
dtype: int64

Extraigo varias muestras de forma aleatoria para verificar otro tipo de nulos

In [86]:
df_hechos.sample(10)

,id,n_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latutid,victima,acusado,gravedad
983,LC-2019-0086214,1,2019-02-10 00:00:00,1.0,No Especificada,NaN,NaN,NaN,NaN,NaN,NaN
3768,LC-2019-0291316,1,2019-05-10 00:00:00,7.0,12,NaN,-58.474902,-34.545690,NaN,NaN,NaN
10669,LC-2020-0595200,2,2020-11-03 00:00:00,15.0,1,AVENIDA,-58.366456,-34.618119,CICLISTA,NaN,NaN
5301,LC-2019-0401653,1,2019-06-28 00:00:00,11.0,5,NaN,-58.414897,-34.637176,NaN,NaN,NaN
10113,LC-2020-0031929,1,2020-01-15 00:00:00,19.0,12,AVENIDA,-58.506262,-34.568242,CICLISTA,AUTO,NaN
12589,LC-2020-0202829,1,2020-04-19 00:00:00,0.0,4,CALLE,-58.366344,-34.638775,MOVIL,NaN,GRAVE
15437,LC-2020-0641333,1,2020-12-03 00:00:00,8.0,3,CALLE,-58.410252,-34.603167,NaN,NaN,NaN
18289,LC-2021-0133217,1,2021-03-23 00:00:00,17.0,12,AVENIDA,-58.492643,-34.569119,MOTO,AUTO,NaN
9191,LC-2019-0727315,1,2019-11-12 00:00:00,11.0,10,NaN,-58.493578,-34.630539,TRANSPORTE PUBLICO,NaN,NaN
5260,LC-2019-0442243,1,2019-06-26 00:00:00,8.0,11,NaN,-58.475523,-34.609725,NaN,NaN,NaN


Los valores nulos en la la columna `gravedad` se remplazan por **leve**
Los valores *NO especificada* en la columna `comuna` se remplazan por valores nulos

In [87]:
df_hechos['gravedad'].fillna('LEVE',inplace=True)
df_hechos['comuna'].replace('No Especificada',np.nan,inplace=True)

#### Cambio de tipo des de datos



Las columnas `franja_hora`, `tipo_calle`, `comuna` y `acusado` serán transformadas al tipo de dato categórico <br>
Las columnas `longitud` y `latitud` serán transformadas al tipo de dato flotante <br>
La columna `fecha` serán transformadas a formato fecha


In [88]:
# cambio a tipo categórico
import pandas as pd
df_hechos['franja_hora'] = df_hechos['franja_hora'].astype('category')
df_hechos['tipo_calle'] = df_hechos['tipo_calle'].astype('category')
df_hechos['comuna'] = df_hechos['comuna'].astype('category')
df_hechos['acusado'] = df_hechos['acusado'].astype('category')

# cambio a flotante los valores de pos x y pos y
df_hechos['longitud'] = df_hechos['longitud'].astype('float')
df_hechos['latutid'] = df_hechos['latutid'].astype('float')

# convierte la columna fecha a tipo fecha
df_hechos['fecha'] = pd.to_datetime(df_hechos['fecha'],errors='coerce')

#### Columnas

##### Renombrar columnas
Se modifican las columnas para mejor lectura y un futuro tratamiento mas cómodo

In [89]:
# Renombrar columnas en df_hechos
df_hechos.rename(columns={'id': 'id_hecho', 'franja_hora': 'hh', 'latutid': 'latitud'}, inplace=True)

En las columnas `victima` y `acusado` se hacen cambios de nombres para coincidir con el csv `homicidios_hechos`.

In [90]:
df_hechos[['acusado', 'victima']] = df_hechos[['acusado', 'victima']].replace({'CICLISTA': 'BICICLETA', 'CARGAS': 'CAMION'})

In [91]:
df_hechos.head()

,id_hecho,n_victimas,fecha,hh,comuna,tipo_calle,longitud,latitud,victima,acusado,gravedad
0,LC-2019-0000179,1,2019-01-01,9.0,14.0,NaN,-58.408911,-34.559658,BICICLETA,NaN,LEVE
1,LC-2019-0000053,1,2019-01-01,1.0,8.0,NaN,-58.443510,-34.669125,AUTO,NaN,LEVE
2,LC-2019-0000063,1,2019-01-01,2.0,8.0,NaN,-58.468335,-34.677556,NaN,NaN,LEVE
3,LC-2019-0000079,1,2019-01-01,2.0,7.0,NaN,-58.437425,-34.647349,PEATON,NaN,LEVE
4,LC-2019-0000082,4,2019-01-01,4.0,3.0,NaN,-58.398225,-34.604579,AUTO,NaN,LEVE


#### Guardo en un CSV para trabajar más cómodo  

In [92]:
df_hechos.to_csv('../Data/lesiones_hechos.csv', index=False)

# Lesiones victimas

Carga del dataset

In [93]:
df_victimas = pd.read_excel('../Data/lesiones.xlsx', sheet_name = 'VICTIMAS', na_values=['SD','sd'])

In [94]:
df_victimas.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,Varon,57.0,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,Varon,NaN,NaN
3,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Varon,45.0,NaN
4,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Mujer,45.0,NaN


Información de las columnas del dataset

In [95]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  11333 non-null  object        
 6   SEXO              25377 non-null  object        
 7   EDAD_VICTIMA      23134 non-null  float64       
 8   GRAVEDAD          537 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 1.9+ MB


#### Elimino columnas innecesarias


Luego de analizar las columnas se determino que: 
* Las columnas `AAAA`,`MM`,`DD` hacen referencia a la información de la columna `FECHA`

In [96]:
df_victimas.drop(columns=['AAA','MM','DD'],inplace=True)

#### Valores nulos
Conteo de nulos

In [97]:
df_victimas.isnull().sum()

ID hecho                0
FECHA                   0
VEHICULO_VICTIMA    16272
SEXO                 2228
EDAD_VICTIMA         4471
GRAVEDAD            27068
dtype: int64

Extraigo varias muestras de forma aleatoria para verificar otro tipo de nulos

In [98]:
df_victimas.sample(10)

,ID hecho,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
8709,LC-2019-0557653,2019-09-05,NaN,Varon,NaN,NaN
16144,LC-2020-0154741,2020-03-10,MOTO,Varon,22.0,NaN
21407,LC-2021-0382173,2021-07-27,NaN,Mujer,42.0,NaN
23135,LC-2021-0246622,2021-06-02,PEATON,Mujer,23.0,NaN
16533,LC-2020-0481838,2020-08-16,MOTO,Mujer,26.0,NaN
3846,LC-2019-0257166,2019-04-24,NaN,Varon,39.0,NaN
15903,LC-2020-0618063,2020-11-17,NaN,Varon,66.0,NaN
7185,LC-2019-0450331,2019-07-20,NaN,Varon,22.0,NaN
19845,LC-2021-0085683,2021-02-25,AUTO,NaN,NaN,NaN
15126,LC-2020-0127087,2020-02-27,CICLISTA,Mujer,40.0,NaN


Los valores nulos presentes en la columna `GRAVEDAD` serán sustituidos por **LEVE**, conforme a las indicaciones proporcionadas en el diccionario de datos, donde se establece que los valores nulos deben ser interpretados como incidentes de gravedad leve

In [99]:
df_victimas['GRAVEDAD'].fillna('LEVE',inplace=True)

#### Cambio de tipo des de datos

Las columnas `VEHICULO_VICTIMA`, `SEXO` y `GRAVEDAD` serán transformadas al tipo de dato categórico


In [100]:
# cambio a tipo categórico
df_victimas['VEHICULO_VICTIMA'] = df_victimas['VEHICULO_VICTIMA'].astype('category')
df_victimas['GRAVEDAD'] = df_victimas['GRAVEDAD'].astype('category')
df_victimas['SEXO'] = df_victimas['SEXO'].astype('category')

#### Columnas

##### Renombrar columnas
Se modifican las columnas para mejor lectura y un futuro tratamiento mas cómodo

In [101]:
# Transformar a minúsculas todas las columnas en lesiones_victimas
df_victimas.columns = df_victimas.columns.str.lower()
# Renombrar columnas en lesiones_victimas
df_victimas.rename(columns={'id hecho': 'id_hecho', 'sexo': 'sexo_victima'}, inplace=True)

Remplazo de la columna `sexo_victima` **VARON** y **MUJER** por `MASCULINO` y `FEMENINO`

In [102]:
df_victimas['sexo_victima'].replace({'Varon':'MASCULINO','Mujer':'FEMENINO'},inplace=True)

In [103]:
df_victimas.head()

,id_hecho,fecha,vehiculo_victima,sexo_victima,edad_victima,gravedad
0,LC-2019-0000053,2019-01-01,NaN,MASCULINO,57.0,LEVE
1,LC-2019-0000063,2019-01-01,NaN,NaN,NaN,LEVE
2,LC-2019-0000079,2019-01-01,NaN,MASCULINO,NaN,LEVE
3,LC-2019-0000082,2019-01-01,NaN,MASCULINO,45.0,LEVE
4,LC-2019-0000082,2019-01-01,NaN,FEMENINO,45.0,LEVE


#### Guardo en un CSV para trabajar más cómodo  

In [104]:
df_victimas.to_csv('../Data/lesiones_victimas.csv', index=False)